In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

1. Architecture Unet

In [8]:
class Unet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear = True):
        super(Unet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear
        
        self.inc = block_doubleConv(n_channels, 64)
        self.down1 = encoder_block(64,128)
        self.down2 = encoder_block(128,256)
        self.down3 = encoder_block(256,512) 
        self.down4 = encoder_block(512,1024) 
        # ajout de 3 couches encoder :
        # self.down5 = encoder_block(1024,1024) 
        # self.down6 = encoder_block(1024,1024) 
        # self.down7 = encoder_block(1024,1024) 
        
        # ajout de 3 couches decoder : 
#         self.up1 = decoder_block(1024, 1024)
#         self.up2 = decoder_block(1024, 1024)
#         self.up3 = decoder_block(1024, 1024)
        self.up1 = decoder_block(1024, 512)
        self.up2 = decoder_block(512, 256)
        self.up3 = decoder_block(256, 128)
        self.up4 = decoder_block(128, 64)
        self.outc = OutConv(64, n_classes)
        
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

2. Block double conv

In [9]:
class block_doubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

3. Encoder block

In [13]:
class encoder_block(nn.Module):
	
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(nn.MaxPool2d(2),
            block_doubleConv(in_channels, out_channels))

    def forward(self, x):
        return self.maxpool_conv(x)

4. Decoder block

In [17]:
class decoder_block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv = block_doubleConv(in_channels, out_channels, in_channels // 2)
        
    def forward(self, x1, x2):
        x1 = self.up(x1)
        
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

5. Block de sortie du Unet

In [ ]:
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

6. Visualisation Unet

In [ ]:
# initialisation d'une entrée
#x = torch.zeros(1, 3, 256, 256, dtype=torch.float, requires_grad=False)
# Chargement du réseau
#test = Unet(x)
# Affo
make_dot(test)